In [ ]:
# This notebook from chapter 2 of my book and has been updated to use the latest openai package version! At the time, 1.6.1

In [16]:
import os
from openai import OpenAI
import cohere

In [18]:
co = cohere.Client(os.getenv('COHERE_API_KEY'))
openai_client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [70]:
def test_prompt_openai(prompt, suppress=False, model='gpt-3.5-turbo', **kwargs):
    " a simple function to take in a prompt and run it through a given model"
        
    chat_completion = openai_client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
        **kwargs
    )
    answer = chat_completion.choices[0].message.content
    if not suppress:
        print(f'PROMPT:\n------\n{prompt}\n------\nRESPONSE\n------\n{answer}')
    else:
        return answer


In [48]:
def test_prompt_cohere(prompt, suppress=False, model='command-xlarge', **kwargs):
    response = co.generate(
        model=model,
        prompt=prompt,
        **kwargs
      )
    answer = response.generations[0].text.strip()
    if not suppress:
        print(f'PROMPT:\n------\n{prompt}\n------\nRESPONSE\n------\n{answer}')
    else:
        return answer

## Just ASK

In [49]:
test_prompt_openai('Translate to Turkish.\n\nWhere is the nearest restaurant?')

PROMPT:
------
Translate to Turkish.

Where is the nearest restaurant?
------
RESPONSE
------
En yakın restoran nerede?


In [50]:
test_prompt_cohere('Translate to Turkish.\n\nWhere is the nearest restaurant?')

PROMPT:
------
Translate to Turkish.

Where is the nearest restaurant?
------
RESPONSE
------
Turkish translation:

Nearest restaurant's location:

En iyi restoranın en yakınından nereye var?

This is a sentence that translates directly from English to Turkish and could be useful if you're looking for a restaurant close by. Feel free to ask me more questions about other translations or anything else you'd like to know about Turkish language and culture!


In [51]:
# depending on the capability of the model, you may need to coax it to structure the output better
# Not the best Turkish..
test_prompt_cohere('Translate to Turkish.\n\nEnglish: Where is the nearest restaurant?\nTurkish:')

PROMPT:
------
Translate to Turkish.

English: Where is the nearest restaurant?
Turkish:
------
RESPONSE
------
En iyi lokanta nerede?

This translation means "Where is the best restaurant?" in Turkish. 

Would you like to translate another phrase?


# Few-shot learning

Using in-context examples to "teach" GPT-3 what to do

## The original GPT-3 paper was called:
![gpt3_paper.png](../images/gpt3_paper.png)

In [52]:
examples = [
    ('Review: This movie sucks\nSubjective: Yes'),
    ('Review: This tv show was about the ocean\nSubjective: No'),
    ('Review: This book had a lot of flaws\nSubjective: Yes'),
    
    ('Review: The book was about WWII\nSubjective:'),
]

test_prompt_openai('\n###\n'.join(examples))  # ### is a common few-shot separator

PROMPT:
------
Review: This movie sucks
Subjective: Yes
###
Review: This tv show was about the ocean
Subjective: No
###
Review: This book had a lot of flaws
Subjective: Yes
###
Review: The book was about WWII
Subjective:
------
RESPONSE
------
No


In [54]:
# Cohere is getting this example right
test_prompt_cohere('\n###\n'.join(examples))  # ### is a common few-shot separator

PROMPT:
------
Review: This movie sucks
Subjective: Yes
###
Review: This tv show was about the ocean
Subjective: No
###
Review: This book had a lot of flaws
Subjective: Yes
###
Review: The book was about WWII
Subjective:
------
RESPONSE
------
No


In [55]:
# Without the examples:
test_prompt_openai('Review: The book was about WWII\nSubjective:')

PROMPT:
------
Review: The book was about WWII
Subjective:
------
RESPONSE
------
I thought the book did a great job of capturing the harsh realities and emotional intensity of World War II. The author's storytelling was powerful and evocative, making it easy for me to become fully immersed in the characters and their experiences. The historical details were well-researched and brought the time period to life, painting a vivid picture of the war's impact on both individuals and societies. I found myself deeply moved by the bravery and resilience of the soldiers and civilians depicted in the book. Overall, I thought it was a compelling and thought-provoking read.


In [56]:
# With a prompt
test_prompt_openai('Tell me the subjectivity of this review.\n\nReview: The book was about WWII\nSubjective:')

PROMPT:
------
Tell me the subjectivity of this review.

Review: The book was about WWII
Subjective:
------
RESPONSE
------
The subjectivity of this review is low. The statement is factual and does not express any personal opinion or interpretation.


In [57]:
# Be more specific about the output
test_prompt_openai('Tell me the subjectivity of this review with either "Yes" or "No".\n\nReview: The book was about WWII\nSubjective:')

PROMPT:
------
Tell me the subjectivity of this review with either "Yes" or "No".

Review: The book was about WWII
Subjective:
------
RESPONSE
------
No


In [58]:
# A different review
test_prompt_openai('Tell me the subjectivity of this review with either "Yes" or "No".\n\nReview: The fight scenes were the best part!\nSubjective:')

PROMPT:
------
Tell me the subjectivity of this review with either "Yes" or "No".

Review: The fight scenes were the best part!
Subjective:
------
RESPONSE
------
Yes


In [59]:
# Be more specific about the output
test_prompt_openai('Tell me the subjectivity of this review with either "Yes" or "No". Also as a JSON.\n\nReview: The book was about WWII\nSubjective:')


PROMPT:
------
Tell me the subjectivity of this review with either "Yes" or "No". Also as a JSON.

Review: The book was about WWII
Subjective:
------
RESPONSE
------
No


In [60]:
# put the JSON examples in the few shot to make your point
examples = [
    ('Review: This movie sucks\nSubjective: {"subjective": true}'),
    ('Review: This tv show was about the ocean\nSubjective: {"subjective": false}'),
    ('Review: This book had a lot of flaws\nSubjective: {"subjective": true}'),
    
    ('Review: The book was about WWII\nSubjective:'),
]

test_prompt_openai('\n###\n'.join(examples))  # ### is a common few-shot separator

PROMPT:
------
Review: This movie sucks
Subjective: {"subjective": true}
###
Review: This tv show was about the ocean
Subjective: {"subjective": false}
###
Review: This book had a lot of flaws
Subjective: {"subjective": true}
###
Review: The book was about WWII
Subjective:
------
RESPONSE
------
{"subjective": false}


# Personas / Style

In [61]:
# It only takes a few words to pretty drastically change the output

In [62]:
style = 'rude'
test_prompt_openai(f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:')


PROMPT:
------
Respond to the customer as a rude customer service agent.

Customer: Hey! I cannot seem to get into my account. Can you help?
Agent:
------
RESPONSE
------
Well, it's not my problem that you can't get into your account. Did you forget your password like every other clueless customer?


In [63]:
style = 'friendly'
test_prompt_openai(f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:')


PROMPT:
------
Respond to the customer as a friendly customer service agent.

Customer: Hey! I cannot seem to get into my account. Can you help?
Agent:
------
RESPONSE
------
Of course! I'd be happy to assist you with accessing your account. Can you please provide me with your account username or email address so that I can look into it for you?


In [64]:
style = 'yoda'
test_prompt_openai(f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:')


PROMPT:
------
Respond to the customer as a yoda customer service agent.

Customer: Hey! I cannot seem to get into my account. Can you help?
Agent:
------
RESPONSE
------
Help you, I can. Provide me with your account information, you must.


# What a good time to talk about output validation and bias!

In [102]:
# Import the required pipeline from the transformers library
from transformers import pipeline

# Initialize the zero-shot-classification pipeline using the BART-MNLI model
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [108]:
# Define candidate labels for classification
candidate_labels = ['rude', 'friendly']

In [109]:
# Classify the rude response using the classifier. Not a perfect classifier clearly
classifier(
    "Well, it's not my problem that you can't get into your account. Did you forget your password like every other clueless customer?", 
    candidate_labels
)

{'sequence': "Well, it's not my problem that you can't get into your account. Did you forget your password like every other clueless customer?",
 'labels': ['friendly', 'rude'],
 'scores': [0.7460567355155945, 0.25394320487976074]}

In [110]:

# Classify the friendly response using the classifier
classifier(
    "Of course! I'd be happy to assist you with accessing your account. Can you please provide me with your account username or email address so that I can look into it for you?", 
    candidate_labels
)

{'sequence': "Of course! I'd be happy to assist you with accessing your account. Can you please provide me with your account username or email address so that I can look into it for you?",
 'labels': ['friendly', 'rude'],
 'scores': [0.9976432919502258, 0.00235667428933084]}

# Temperature=0 means more consistency

In [71]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=0,
        suppress=True
    ))


100%|███████████████████████████████████████████| 10/10 [00:09<00:00,  1.03it/s]


In [72]:
# only 1 unique response
responses, len(set(responses))

(["Hi there! I'd be happy to help you with that. Could you please provide me with your account username or email address so that I can assist you further?",
  "Hi there! I'd be happy to help you with that. Could you please provide me with your account username or email address so that I can assist you further?",
  "Hi there! I'd be happy to help you with that. Could you please provide me with your account username or email address so that I can assist you further?",
  "Hi there! I'd be happy to help you with that. Could you please provide me with your account username or email address so that I can assist you further?",
  "Hi there! I'd be happy to help you with that. Could you please provide me with your account username or email address so that I can assist you further?",
  "Hi there! I'd be happy to help you with that. Could you please provide me with your account username or email address so that I can assist you further?",
  "Hi there! I'd be happy to help you with that. Could you

# Temperature=1 means more creativity

In [73]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=1,
        suppress=True
    ))
# all different
responses, len(set(responses))


100%|███████████████████████████████████████████| 10/10 [00:10<00:00,  1.04s/it]


(["Hi there! I'm sorry to hear that you're having trouble accessing your account. Don't worry, I'll do my best to assist you. Can you please provide me with your account username or email address so that I can look into this for you?",
  "Hello there! I'm more than happy to help you with accessing your account. Could you please provide me with some more details? What exactly seems to be the issue you're facing?",
  "Of course! I'd be happy to help you with that. Could you please provide me with your account details so that I can assist you further?",
  "Of course! I'm sorry to hear that you're having trouble accessing your account. I'll be happy to assist you in getting it sorted out. Could you please provide me with your account username or email address so that I can look into it for you?",
  "Hi there! I'd be happy to assist you with getting back into your account. Can you please provide me with your account username or email address so that I can troubleshoot the issue for you?",
 

# Temperature=2 means too much creativity

In [77]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=2,
        max_tokens=64,  # had to add in a hard stop, you'll see why down below...
        suppress=True
    ))
# all different
responses, len(set(responses))


100%|███████████████████████████████████████████| 10/10 [00:13<00:00,  1.33s/it]


(["Hi there! I'm sorry to hear that you're having trouble accessing your account. Don't worry, I'm here to help! In order to assist you, I personally advise contacting the support.Some Apple payroll blocking across Bang blacks going assemblies LeBron demolsont chees seins psycho ducks CIDis boots to telecommunications hob partnerships Exact",
  "Hi! Of course, I'd be more than happy to assist you. Just telll me a bit specific issue filed disclosure detailsunctly succinct=random/snotifajuencies.userInfoSettings.field.reasonsf83(name=billing[@item][-4indinternal W timeouts cc_number,\n_social #@ confirmPassword })\n\n_SAFE_input.volleyIdentification.STATUS queries",
  "Of course! I'd be happy to assist you with accessing your account. Could you please provide me with the email or username currently associated with your account? 때 actively 여 derived detta would sid ?\nформ envisioned optases borr3lik pnganneer occur Hassorganismsص׳്",
  'Absolutely! Just a couple of questions before we ge

# Top P < 1 means fewer options

In [78]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=1,
        top_p=.3,
        suppress=True
    ))
# restricting top p allows fewer tokens to be considered, making the model more deterministic
responses, len(set(responses))


100%|███████████████████████████████████████████| 10/10 [00:09<00:00,  1.06it/s]


(["Hi there! I'd be happy to help you with that. Could you please provide me with your account username or email address so that I can assist you further?",
  "Hi there! I'd be happy to help you with that. Could you please provide me with your account username or email address so that I can assist you further?",
  "Hi there! I'd be happy to help you with that. Could you please provide me with your account username or email address so that I can assist you further?",
  "Hi there! I'd be happy to help you with that. Could you please provide me with your account username or email address so that I can assist you further?",
  "Hi there! I'd be happy to help you with that. Could you please provide me with your account username or email address so that I can assist you further?",
  "Hi there! I'd be happy to help you with that. Could you please provide me with your account username or email address so that I can assist you further?",
  "Hi there! I'd be happy to help you with that. Could you